In [0]:
from google.colab import files
data=files.upload()

Saving train.csv to train (1).csv


In [0]:
#Importing libraries
import numpy as np
import tensorflow as tf
import pandas as pd
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import sklearn
from sklearn import model_selection

In [74]:
#Loading dataset
df=pd.read_csv('train.csv').values
print(df)

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [7 0 0 ... 0 0 0]
 [6 0 0 ... 0 0 0]
 [9 0 0 ... 0 0 0]]


In [0]:
#Splitting datset into train and cross validation set
X_train = df[:35000, 1:].reshape(35000,28, 28,1).astype( 'float32' )
X_test=df[35001:,1:].reshape(df.shape[0]-35000-1,28, 28,1).astype( 'float32' )
X_train = X_train/ 255.0
X_test=X_test/255.0
Y_train = df[:35000,0]
Y_test=df[35001:,0]

In [0]:

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()

Y_train = lb.fit_transform(Y_train)
Y_test = lb.fit_transform(Y_test)

In [77]:
#Printing shapes 
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(35000, 28, 28, 1)
(35000, 10)
(6999, 28, 28, 1)
(6999, 10)


In [0]:
#Model
def cnn(input_shape):

    
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
    
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool1')(X)
    
    X = Conv2D(64, (5, 5), padding='valid',strides = (1, 1), name = 'conv1')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(128, (3, 3), strides = (1, 1), name = 'conv2')(X)
    X = Activation('relu')(X)
    X=MaxPooling2D((5, 5), name='max_pool2')(X)
    X = Flatten()(X)
    #Three fully connected layers
    X=Dense(10000, activation='relu', name='fc1',kernel_regularizer=regularizers.l2(0.01))(X)
    X = Dense(100, activation='relu', name='fc2',kernel_regularizer=regularizers.l2(0.01))(X)
    X = Dense(10, activation='relu', name='fc3')(X)
    
    model = Model(inputs = X_input, outputs = X, name='HappyModel')

    
    return model

In [0]:
from keras import regularizers
model = cnn(X_train[1].shape)

In [0]:
model.compile(optimizer = 'Adam', loss ="mean_squared_error", metrics = ["accuracy"])

In [108]:
model.fit(x =X_train, y =Y_train, epochs = 10, batch_size = 16)

Epoch 1/10
16176/35000 [============>.................] - ETA: 4:02 - loss: 0.0067 - acc: 0.980034304/35000 [============================>.] - ETA: 8s - loss: 0.0064 - acc: 0.981335000/35000 [==============================] - 451s 13ms/step - loss: 0.0064 - acc: 0.9812
Epoch 2/10
 7760/35000 [=====>........................] - ETA: 5:49 - loss: 0.0067 - acc: 0.98025776/35000 [=====================>........] - ETA: 1:58 - loss: 0.0065 - acc: 0.981635000/35000 [==============================] - 450s 13ms/step - loss: 0.0063 - acc: 0.9821
Epoch 3/10
 3888/35000 [==>...........................] - ETA: 6:39 - loss: 0.0051 - acc: 0.987722224/35000 [==================>...........] - ETA: 2:45 - loss: 0.0065 - acc: 0.982735000/35000 [==============================] - 452s 13ms/step - loss: 0.0063 - acc: 0.9831
Epoch 4/10
 2416/35000 [=>............................] - ETA: 6:59 - loss: 0.0062 - acc: 0.983020560/35000 [================>.............] - ETA: 3:07 - loss: 0.0057 - acc: 0.984335000/

In [109]:
#Testing the model on cross validation set
preds =model.evaluate(x = X_test, y = Y_test)
print(preds)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

6999/6999 [==============================] - 13s 2ms/step
[0.005432287841723389, 0.9821403057579654]
Loss = 0.005432287841723389
Test Accuracy = 0.9821403057579654
